In [ ]:
import gdown
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pprint
import random
import pandas as pd
import pulp
import itertools
import requests
import random
from geopy.distance import geodesic
import time
import pandas as pd
import os

Downloading...
From (original): https://drive.google.com/uc?id=1zq17RENYLU1qrE2gNPsxRAPoysWMZAqf
From (redirected): https://drive.google.com/uc?id=1zq17RENYLU1qrE2gNPsxRAPoysWMZAqf&confirm=t&uuid=8aa9e8e0-966a-4d51-83bf-feb55dd68e17
To: /content/mpsis_setup.py
100%|██████████| 12.1k/12.1k [00:00<00:00, 21.9MB/s]


Mounted at /content/drive


In [ ]:
def generate_random_point(sw_lat, sw_lng, ne_lat, ne_lng):
    lat = random.uniform(sw_lat, ne_lat)
    lng = random.uniform(sw_lng, ne_lng)
    return lat, lng

def snap_to_road(lat, lng, api_key):
    url = f'https://roads.googleapis.com/v1/nearestRoads?points={lat},{lng}&key={api_key}'
    response = requests.get(url)
    data = response.json()
    if 'snappedPoints' in data and len(data['snappedPoints']) > 0:
        snapped_point = data['snappedPoints'][0]['location']
        return snapped_point['latitude'], snapped_point['longitude']
    return None

def get_road_distance_time(origin, destination, api_key):
    origin_str = f'{origin[0]}, {origin[1]}'
    destination_str = f'{destination[0]}, {destination[1]}'
    departure_time = int(time.time()) + 24 * 3600
    url = (
        f'https://maps.googleapis.com/maps/api/distancematrix/json?units=metric&origins={origin_str}'
        f'&destinations={destination_str}&mode=driving&departure_time={departure_time}&key={api_key}'
    )
    response = requests.get(url)
    data = response.json()

    if data['status'] == 'OK':
        element = data['rows'][0]['elements'][0]
        if element['status'] == 'OK':
            distance_text = element['distance']['text']
            distance = float(distance_text.replace(' km', '').replace(',', ''))

            duration_text = element['duration']['text']
            duration_parts = duration_text.split()
            duration = int(duration_parts[0])
            if "hour" in duration_text:
                duration = int(duration_parts[0]) * 60 + int(duration_parts[2])

            return distance, duration
        else:
            return None, None
    else:
        return None, None

def find_center(sw_lat, sw_lng, ne_lat, ne_lng):
    center_lat = (sw_lat + ne_lat) / 2
    center_lng = (sw_lng + ne_lng) / 2
    return center_lat, center_lng

def generate_list(n, target_sum):
    if n <= 1:
        raise ValueError("The length of the list must be greater than 1.")
    if target_sum < 0:
        raise ValueError("The target sum must be non-negative.")
    result = [0]
    random_values = np.random.rand(n - 1)
    random_values *= target_sum / np.sum(random_values)
    rounded_values = np.round(random_values, 2)
    difference = target_sum - np.sum(rounded_values)

    if len(rounded_values) > 0:
        rounded_values[-1] += difference

    result.extend(rounded_values)

    return result

def crop_matrix(data, N):
    return [row[:N] for row in data[:N]]

import numpy as np

def generate_demand_list(n, m, target_sum):
    if n <= 1:
        raise ValueError("The length of the list must be greater than 1.")
    if m >= n:
        raise ValueError("M must be less than N.")
    if target_sum < 0:
        raise ValueError("The target sum must be non-negative.")

    random_values = np.random.rand(n - m - 1)
    random_values *= target_sum / np.sum(random_values)
    rounded_values = np.round(random_values, 2)
    difference = target_sum - np.sum(rounded_values)

    if len(rounded_values) > 0:
        rounded_values[-1] += difference

    zeros_list = [0] * m

    result = list(rounded_values) + zeros_list

    random.shuffle(result)

    result = [-1] + result

    return result
def replace_zeros_with_sum(input_list, target_sum):
    zero_positions = [i for i, value in enumerate(input_list) if value == 0]
    num_zeros = len(zero_positions)

    result = input_list.copy()

    if num_zeros == 0:
        raise ValueError("There are no zeros to replace in the list.")

    random_values = np.random.rand(num_zeros)
    random_values *= target_sum / np.sum(random_values)
    rounded_values = np.round(random_values, 2)
    difference = target_sum - np.sum(rounded_values)

    if len(rounded_values) > 0:
        rounded_values[-1] += difference

    for i, pos in enumerate(zero_positions):
        result[pos] = rounded_values[i]

    return result

def create_path_as_list(pairs_list):
    wanted_length = len(pairs_list)
    path = ''
    current_pair = next(pair for pair in pairs_list if pair[0] == 0)
    path += str(current_pair[0])

    next_start = current_pair[1]

    pairs_list.remove(current_pair)

    while len(path) < wanted_length:
        path += str(next_start)
        current_pair = next(pair for pair in pairs_list if pair[0] == next_start)
        pairs_list.remove(current_pair)
        next_start = current_pair[1]

    return path

def find_paths_vrb(edges, k):
    def find_path_vrb(current_path):
        if len(paths) >= k:
            return

        last_node = current_path[-1]

        if last_node == 0 and len(current_path) > 1:
            paths.append(current_path)
            return

        for edge in edges:
            if edge[0] == last_node:
                find_path_vrb(current_path + [edge[1]])

    paths = []
    find_path_vrb([0])

    return paths

def repetitions(list1, list2):
  common_elements = set(list1) & set(list2)
  return (2*len(common_elements))/len(list1+list2)

def equal_paths(list1, list2):
  are_equal = sorted(list1) == sorted(list2)
  if are_equal:
    return 1
  else:
    return 0

In [ ]:
def create_cost_matrixes(df):
    real_distance_columns = [col for col in df.columns if col.startswith('Road Distance to Point')]
    df_filtered = df[real_distance_columns]

    num_points = len(df_filtered)
    cost_raod = [[0] * num_points for _ in range(num_points)]

    for i in range(num_points):
        for j in range(num_points):
            if i != j:
                column_name = f'Road Distance to Point {j + 1}'
                distance = df_filtered.iloc[i][column_name]

                cost_raod[i][j] = float(distance)


    real_distance_columns = [col for col in df.columns if col.startswith('Time to Point')]
    df_filtered = df[real_distance_columns]

    num_points = len(df_filtered)
    cost_time = [[0] * num_points for _ in range(num_points)]

    for i in range(num_points):
        for j in range(num_points):
            if i != j:
                column_name = f'Time to Point {j + 1}'
                distance = df_filtered.iloc[i][column_name]

                cost_time[i][j] = float(distance)


    real_distance_columns = [col for col in df.columns if col.startswith('Real Distance to Point')]
    df_filtered = df[real_distance_columns]

    num_points = len(df_filtered)
    cost_real = [[0] * num_points for _ in range(num_points)]

    for i in range(num_points):
        for j in range(num_points):
            if i != j:
                column_name = f'Real Distance to Point {j + 1}'
                distance = df_filtered.iloc[i][column_name]

                cost_real[i][j] = float(distance)

    return cost_raod, cost_time, cost_real

In [ ]:
def find_paths_vrb(edges, k, depot_one_more_time):
    def find_path_vrb(current_path):
        if len(paths) >= k:
            return

        last_node = current_path[-1]

        if last_node == depot_one_more_time and len(current_path) > 1:
            paths.append(current_path)
            return

        for edge in edges:
            if edge[0] == last_node:
                find_path_vrb(current_path + [edge[1]])

    paths = []
    find_path_vrb([0])
    return paths

def vrptw(number_of_clients, number_of_vehicles, cost_buffor, a, b, t, T,  file_idx, problem_idx):
    try:
      n = number_of_clients
      c = number_of_vehicles
      Q = 100
      M = 10000000

      cost_shiffted = crop_matrix(cost_buffor,n+1)
      cost_shiffted.append(cost_shiffted[0])
      iterator = 0
      for row in cost_shiffted:
        cost_shiffted[iterator].append(cost_shiffted[iterator][0])
        iterator += 1

      prob = pulp.LpProblem("VRPTW", pulp.LpMinimize)

      x = pulp.LpVariable.dicts("x", [(i,j,k) for i in range(n+1) for j in range(1, n+2) for k in range(c)], cat='Binary')
      s = pulp.LpVariable.dicts("s", [(i,k) for i in range(n+2) for k in range(c)], lowBound=0, cat='Continuous')

      prob += pulp.lpSum(cost_shiffted[i][j] * x[(i,j,k)] for i in range(n+1) for j in range(1, n+2) for k in range(c))

      for j in range(1, n+1):
          prob += pulp.lpSum(x[(i,j,k)] for i in range(n+1) for k in range(c)) == 1

      for i in range(1, n+1):
          prob += pulp.lpSum(x[(i,j,k)] for j in range(1, n+2) for k in range(c)) == 1

      for k in range(c):
          prob += pulp.lpSum(x[i,n+1,k] for i in range(n+1)) == 1

      for k in range(c):
          prob += pulp.lpSum(x[0,j,k] for j in range(1, n+2)) == 1

      for m in range(1, n+1):
        for k in range(c):
          prob += pulp.lpSum(x[(i,m,k)] for i in range(n+1) if i != m) == pulp.lpSum(x[(m,j,k)] for j in range(1,n+2) if j != m)

      for k in range(c):
        prob += x[0,n+1,k] == 0

      for i in range(n+1):
        for j in range(1,n+2):
          for k in range(c):
            if i == j:
              prob += x[(i,j,k)] == 0

      for i in range(n+1):
        for j in range(1,n+2):
          for k in range(c):
            prob += s[(i,k)] + t[i] + T[i][j] <= s[(j,k)] + M*(1 - x[(i,j,k)])

      for i in range(1, n+1):
        for k in range(c):
          prob += M*(pulp.lpSum(x[i,j,k] for j in range(1,n+2)) - 1) + a[i] <= s[(i,k)]

      for i in range(1, n+1):
        for k in range(c):
          prob += s[(i,k)] + t[i] <= b[i] + M*(1 - pulp.lpSum(x[(i,j,k)] for j in range(1,n+2)))

      prob.solve()

      print(pulp.value(prob.objective))

      all_uesd_edges = []
      paths = [[] for _ in range(c)]

      for k in range(c):
          for i in range(n + 1):
              for j in range(1, n + 2):
                  if i != j and pulp.value(x[i,j,k]) == 1:
                      all_uesd_edges.append((i, j))
                      paths[k].append((i, j))

      result_paths = find_paths_vrb(all_uesd_edges, number_of_clients, number_of_clients+1)
      return pulp.value(prob.objective), result_paths, all_uesd_edges

    except Exception as e:
        print(f"error for delivery stops {number_of_clients} and vehicles {number_of_vehicles}")

        values_to_reload.append((number_of_clients, number_of_vehicles, file_idx, problem_idx))

        return None, None, None

def vrptw_error(number_of_clients, number_of_vehicles, cost_buffor, a, b, t, T, file_idx):
  try:
      n = number_of_clients
      c = number_of_vehicles
      Q = 100
      M = 10000000

      cost_shiffted = crop_matrix(cost_buffor,n+1)
      cost_shiffted.append(cost_shiffted[0])
      iterator = 0
      for row in cost_shiffted:
        cost_shiffted[iterator].append(cost_shiffted[iterator][0])
        iterator += 1

      prob = pulp.LpProblem("VRPTW", pulp.LpMinimize)

      x = pulp.LpVariable.dicts("x", [(i,j,k) for i in range(n+1) for j in range(1, n+2) for k in range(c)], cat='Binary')
      s = pulp.LpVariable.dicts("s", [(i,k) for i in range(n+2) for k in range(c)], lowBound=0, cat='Continuous')

      prob += pulp.lpSum(cost_shiffted[i][j] * x[(i,j,k)] for i in range(n+1) for j in range(1, n+2) for k in range(c))

      for j in range(1, n+1):
          prob += pulp.lpSum(x[(i,j,k)] for i in range(n+1) for k in range(c)) == 1

      for i in range(1, n+1):
          prob += pulp.lpSum(x[(i,j,k)] for j in range(1, n+2) for k in range(c)) == 1

      for k in range(c):
          prob += pulp.lpSum(x[i,n+1,k] for i in range(n+1)) == 1

      for k in range(c):
          prob += pulp.lpSum(x[0,j,k] for j in range(1, n+2)) == 1

      for m in range(1, n+1):
        for k in range(c):
          prob += pulp.lpSum(x[(i,m,k)] for i in range(n+1) if i != m) == pulp.lpSum(x[(m,j,k)] for j in range(1,n+2) if j != m)

      for k in range(c):
        prob += x[0,n+1,k] == 0

      for i in range(n+1):
        for j in range(1,n+2):
          for k in range(c):
            if i == j:
              prob += x[(i,j,k)] == 0

      for i in range(n+1):
        for j in range(1,n+2):
          for k in range(c):
            prob += s[(i,k)] + t[i] + T[i][j] <= s[(j,k)] + M*(1 - x[(i,j,k)])

      for i in range(1, n+1):
        for k in range(c):
          prob += M*(pulp.lpSum(x[i,j,k] for j in range(1,n+2)) - 1) + a[i] <= s[(i,k)]

      for i in range(1, n+1):
        for k in range(c):
          prob += s[(i,k)] + t[i] <= b[i] + M*(1 - pulp.lpSum(x[(i,j,k)] for j in range(1,n+2)))

      prob.solve()

      print(pulp.value(prob.objective))

      all_uesd_edges = []
      paths = [[] for _ in range(c)]

      for k in range(c):
          for i in range(n + 1):
              for j in range(1, n + 2):
                  if i != j and pulp.value(x[i,j,k]) == 1:
                      all_uesd_edges.append((i, j))
                      paths[k].append((i, j))

      result = find_paths_vrb(all_uesd_edges, number_of_clients, number_of_clients+1)
      return pulp.value(vrpsdp.objective), result_paths, all_uesd_edges

  except Exception as e:
      print(f"ONE MORE TIME ERROR")
      return None, None, None

In [4]:
import math
import pandas as pd
import os
service_time = 5
number_of_clients = 10
number_of_vehicles = 2
output_dir = 'X'
df = pd.read_csv(os.path.join(output_dir, "X"))
df.fillna(10000000, inplace=True)
cost_R, cost_T, cost_G = create_cost_matrixes(df)

T = crop_matrix(cost_T,number_of_clients+1)
median = np.median(T)
max_time = math.floor(number_of_clients * median + service_time*number_of_clients/number_of_vehicles) +1

time_window_time_list = list(range(50,2701,50))

t = [service_time] * (number_of_clients + 10)
a = [0] * (number_of_clients + 1)
b = [max_time] * (number_of_clients + 1)


number_of_trials = 20
time_window_time_list = list(range(50,2701,50))

cost =  [[]  for _ in time_window_time_list]

for iterator in range(number_of_trials):
  print(f"######################################### Number of trial {iterator}")

  for client in range(number_of_clients + 1):
      a[client] = random.randint(int(median/2), int(max_time - service_time - median))

  for time_window_time in time_window_time_list:
      print(f"time window value {time_window_time}")

      for client in range(number_of_clients + 1):
        b[client] = a[client] + time_window_time


      obj = vrptw(number_of_clients, number_of_vehicles, cost_R, a, b, t, T)

      cost[time_window_time_list.index(time_window_time)].append(obj)

In [2]:
# # print(cost)
# cost = [['INFEISABLE', 'INFEISABLE', 'INFEISABLE', 'INFEISABLE', 'INFEISABLE', 'INFEISABLE', 'INFEISABLE', 'INFEISABLE', 'INFEISABLE', 'INFEISABLE', 'INFEISABLE', 'INFEISABLE', 'INFEISABLE', 'INFEISABLE', 'INFEISABLE', 'INFEISABLE', 'INFEISABLE', 'INFEISABLE', 'INFEISABLE', 'INFEISABLE'], ['INFEISABLE', 'INFEISABLE', 'INFEISABLE', 'INFEISABLE', 'INFEISABLE', 'INFEISABLE', 'INFEISABLE', 'INFEISABLE', 'INFEISABLE', 'INFEISABLE', 'INFEISABLE', 'INFEISABLE', 'INFEISABLE', 'INFEISABLE', 'INFEISABLE', 'INFEISABLE', 'INFEISABLE', 'INFEISABLE', 'INFEISABLE', 'INFEISABLE'], ['INFEISABLE', 'INFEISABLE', 'INFEISABLE', 'INFEISABLE', 'INFEISABLE', 'INFEISABLE', 'INFEISABLE', 'INFEISABLE', 'INFEISABLE', 'INFEISABLE', 67.3, 'INFEISABLE', 'INFEISABLE', 'INFEISABLE', 'INFEISABLE', 'INFEISABLE', 'INFEISABLE', 'INFEISABLE', 'INFEISABLE', 'INFEISABLE'], ['INFEISABLE', 'INFEISABLE', 'INFEISABLE', 'INFEISABLE', 'INFEISABLE', 'INFEISABLE', 61.99999999999999, 'INFEISABLE', 'INFEISABLE', 'INFEISABLE', 60.4, 'INFEISABLE', 'INFEISABLE', 'INFEISABLE', 'INFEISABLE', 'INFEISABLE', 'INFEISABLE', 'INFEISABLE', 'INFEISABLE', 'INFEISABLE'], ['INFEISABLE', 'INFEISABLE', 'INFEISABLE', 'INFEISABLE', 57.6, 71.2, 61.99999999999999, 'INFEISABLE', 'INFEISABLE', 'INFEISABLE', 63.4, 'INFEISABLE', 60.0, 'INFEISABLE', 'INFEISABLE', 64.3, 'INFEISABLE', 'INFEISABLE', 'INFEISABLE', 'INFEISABLE'], ['INFEISABLE', 'INFEISABLE', 'INFEISABLE', 'INFEISABLE', 57.6, 66.8, 61.99999999999999, 'INFEISABLE', 65.89999999999999, 'INFEISABLE', 60.4, 77.4, 62.10000000000001, 'INFEISABLE', 'INFEISABLE', 64.3, 76.3, 63.4, 82.3, 67.0], ['INFEISABLE', 64.2, 63.6, 69.49999999999999, 67.30000000000001, 66.0, 66.0, 'INFEISABLE', 65.7, 'INFEISABLE', 59.6, 72.50000000000001, 60.800000000000004, 77.19999999999999, 66.10000000000001, 62.10000000000001, 75.9, 63.70000000000001, 82.3, 66.0], [62.0, 62.8, 62.0, 60.5, 62.49999999999999, 78.49999999999999, 61.50000000000001, 'INFEISABLE', 64.80000000000001, 'INFEISABLE', 60.4, 74.60000000000001, 61.60000000000001, 77.0, 62.800000000000004, 64.3, 69.5, 66.6, 79.39999999999998, 65.5], [65.7, 57.900000000000006, 62.0, 60.5, 55.60000000000001, 64.89999999999999, 60.300000000000004, 'INFEISABLE', 57.8, 'INFEISABLE', 59.6, 73.7, 59.70000000000001, 'INFEISABLE', 64.0, 62.10000000000001, 64.39999999999999, 62.2, 68.89999999999999, 60.800000000000004], [59.90000000000001, 57.800000000000004, 55.3, 60.5, 55.60000000000001, 65.49999999999999, 60.1, 70.6, 57.300000000000004, 63.3, 59.6, 67.3, 59.80000000000001, 71.9, 62.800000000000004, 62.8, 64.39999999999999, 56.9, 65.39999999999999, 63.1], [61.50000000000001, 57.800000000000004, 55.900000000000006, 60.5, 55.60000000000001, 64.89999999999999, 61.99999999999999, 69.6, 57.300000000000004, 63.3, 55.99999999999999, 67.3, 53.10000000000001, 67.1, 56.900000000000006, 53.60000000000001, 66.39999999999999, 56.7, 64.5, 59.699999999999996], [59.7, 57.699999999999996, 55.3, 60.5, 56.9, 65.49999999999999, 57.90000000000001, 63.400000000000006, 54.4, 63.99999999999999, 54.9, 67.4, 53.10000000000001, 66.8, 60.40000000000001, 53.60000000000001, 63.8, 56.7, 62.1, 59.699999999999996], [59.8, 57.699999999999996, 55.3, 60.5, 54.900000000000006, 63.79999999999999, 60.199999999999996, 63.400000000000006, 54.4, 61.2, 53.99999999999999, 63.7, 53.10000000000001, 65.6, 56.70000000000001, 54.50000000000001, 61.50000000000001, 56.4, 55.2, 59.599999999999994], [57.600000000000016, 52.900000000000006, 55.3, 60.5, 55.800000000000004, 63.9, 55.00000000000001, 62.9, 54.4, 61.89999999999999, 53.99999999999999, 61.2, 53.3, 64.5, 56.70000000000001, 53.80000000000001, 62.300000000000004, 56.10000000000001, 54.3, 59.3], [57.600000000000016, 51.30000000000001, 56.5, 60.6, 55.800000000000004, 61.300000000000004, 55.00000000000001, 62.9, 54.4, 62.699999999999996, 53.99999999999999, 59.7, 53.3, 63.5, 53.60000000000001, 54.50000000000001, 63.400000000000006, 56.70000000000001, 53.60000000000001, 55.9], [57.00000000000001, 51.30000000000001, 56.20000000000001, 60.5, 59.3, 61.300000000000004, 52.00000000000001, 61.10000000000001, 53.9, 61.2, 50.20000000000001, 58.5, 51.50000000000001, 63.5, 55.500000000000014, 53.60000000000001, 61.50000000000001, 56.70000000000001, 53.60000000000001, 53.900000000000006], [55.70000000000001, 51.30000000000001, 55.900000000000006, 60.1, 55.4, 60.20000000000001, 52.6, 60.900000000000006, 52.9, 61.2, 50.20000000000001, 58.2, 50.20000000000001, 64.2, 53.60000000000001, 53.60000000000001, 62.300000000000004, 55.8, 53.60000000000001, 58.40000000000001], [56.600000000000016, 51.6, 55.3, 60.1, 55.4, 60.20000000000001, 50.9, 54.100000000000016, 52.9, 61.89999999999999, 50.20000000000001, 58.2, 51.0, 56.1, 53.60000000000001, 53.30000000000001, 61.50000000000001, 56.00000000000001, 54.0, 55.7], [54.300000000000004, 52.900000000000006, 56.2, 60.1, 54.5, 60.20000000000001, 50.00000000000001, 52.70000000000001, 52.9, 62.2, 50.20000000000001, 58.2, 52.00000000000001, 55.699999999999996, 53.60000000000001, 53.30000000000001, 62.300000000000004, 58.0, 53.599999999999994, 55.7], [54.300000000000004, 52.00000000000001, 54.599999999999994, 58.1, 55.300000000000004, 59.60000000000001, 50.9, 52.70000000000001, 52.9, 61.2, 50.20000000000001, 56.2, 50.20000000000001, 53.1, 55.6, 53.80000000000001, 61.50000000000001, 55.400000000000006, 53.4, 53.900000000000006], [55.70000000000001, 51.30000000000001, 55.8, 57.3, 55.300000000000004, 58.80000000000001, 52.300000000000004, 52.70000000000001, 53.49999999999999, 60.5, 50.20000000000001, 58.2, 51.0, 55.699999999999996, 55.2, 53.60000000000001, 60.900000000000006, 54.900000000000006, 53.4, 53.900000000000006], [54.80000000000001, 51.6, 52.6, 57.3, 55.60000000000001, 53.7, 50.9, 52.900000000000006, 53.10000000000001, 60.5, 50.20000000000001, 56.2, 52.2, 53.1, 55.2, 53.60000000000001, 61.900000000000006, 55.60000000000001, 53.5, 53.2], [54.300000000000004, 51.30000000000001, 53.6, 58.1, 54.5, 53.400000000000006, 50.00000000000001, 52.70000000000001, 51.10000000000001, 60.5, 50.20000000000001, 57.10000000000001, 50.20000000000001, 55.0, 54.9, 52.00000000000001, 60.900000000000006, 55.1, 53.4, 52.6], [57.50000000000001, 51.1, 52.20000000000001, 57.3, 54.7, 53.0, 50.9, 52.70000000000001, 51.10000000000001, 59.800000000000004, 50.20000000000001, 57.10000000000001, 50.20000000000001, 54.699999999999996, 54.70000000000001, 53.60000000000001, 60.10000000000001, 52.900000000000006, 53.3, 52.6], [58.00000000000001, 50.20000000000001, 52.20000000000001, 57.3, 53.80000000000001, 53.400000000000006, 50.50000000000001, 52.900000000000006, 51.1, 59.5, 51.300000000000004, 56.2, 50.20000000000001, 54.00000000000001, 53.60000000000001, 52.60000000000001, 61.1, 52.900000000000006, 53.599999999999994, 52.6], [54.80000000000001, 50.20000000000001, 52.6, 58.1, 54.4, 53.0, 50.50000000000001, 52.70000000000001, 51.10000000000001, 58.5, 50.20000000000001, 58.5, 50.20000000000001, 53.1, 53.60000000000001, 52.00000000000001, 60.8, 52.900000000000006, 55.900000000000006, 52.4], [54.300000000000004, 50.20000000000001, 52.00000000000001, 57.3, 53.80000000000001, 53.10000000000001, 50.00000000000001, 52.70000000000001, 51.1, 58.5, 50.20000000000001, 56.2, 50.00000000000001, 54.6, 54.70000000000001, 52.00000000000001, 58.1, 52.900000000000006, 51.1, 51.1], [54.300000000000004, 50.20000000000001, 52.800000000000004, 58.1, 54.7, 53.0, 50.50000000000001, 52.800000000000004, 51.1, 53.699999999999996, 50.20000000000001, 56.2, 50.20000000000001, 53.1, 52.4, 52.00000000000001, 57.9, 53.0, 51.1, 51.1], [54.300000000000004, 51.0, 51.1, 53.10000000000001, 51.900000000000006, 53.4, 50.50000000000001, 52.70000000000001, 51.1, 53.1, 51.0, 56.2, 50.20000000000001, 53.800000000000004, 52.9, 52.00000000000001, 56.2, 51.0, 51.1, 51.6], [53.900000000000006, 51.1, 51.1, 50.00000000000001, 53.2, 53.400000000000006, 52.50000000000001, 52.70000000000001, 51.1, 52.699999999999996, 52.8, 53.0, 50.00000000000001, 54.1, 52.4, 51.1, 57.10000000000001, 51.0, 51.1, 51.0], [53.40000000000001, 50.20000000000001, 52.00000000000001, 50.00000000000001, 51.0, 53.3, 50.9, 53.1, 51.10000000000001, 54.50000000000001, 50.20000000000001, 52.6, 50.20000000000001, 53.1, 53.60000000000001, 51.1, 56.2, 51.1, 51.1, 51.1], [52.00000000000001, 50.20000000000001, 56.7, 51.1, 55.30000000000001, 53.0, 50.00000000000001, 51.0, 51.10000000000001, 52.0, 50.20000000000001, 53.00000000000001, 50.00000000000001, 53.1, 52.6, 50.20000000000001, 56.199999999999996, 50.20000000000001, 52.00000000000001, 51.1], [51.1, 51.0, 51.1, 50.50000000000001, 50.20000000000001, 52.70000000000001, 50.00000000000001, 51.0, 51.1, 53.4, 50.20000000000001, 52.00000000000001, 51.0, 53.1, 52.4, 50.20000000000001, 56.199999999999996, 50.20000000000001, 50.9, 50.20000000000001], [52.00000000000001, 50.20000000000001, 51.1, 50.50000000000001, 50.20000000000001, 52.7, 50.9, 51.0, 51.1, 52.6, 50.20000000000001, 53.1, 50.00000000000001, 53.1, 52.4, 50.20000000000001, 56.199999999999996, 50.20000000000001, 50.20000000000001, 50.20000000000001], [51.1, 50.20000000000001, 51.300000000000004, 50.00000000000001, 50.20000000000001, 51.1, 50.20000000000001, 52.0, 51.1, 52.2, 50.20000000000001, 52.8, 50.00000000000001, 52.900000000000006, 52.4, 50.20000000000001, 52.199999999999996, 50.20000000000001, 50.20000000000001, 51.1], [51.1, 51.30000000000001, 51.1, 50.50000000000001, 50.20000000000001, 50.20000000000001, 50.9, 50.20000000000001, 50.20000000000001, 52.00000000000001, 50.20000000000001, 52.00000000000001, 50.00000000000001, 52.900000000000006, 52.4, 50.20000000000001, 52.199999999999996, 50.20000000000001, 50.20000000000001, 51.1], [51.300000000000004, 50.20000000000001, 51.1, 50.50000000000001, 50.20000000000001, 50.20000000000001, 50.00000000000001, 51.1, 50.20000000000001, 52.00000000000001, 50.20000000000001, 51.10000000000001, 50.20000000000001, 53.1, 52.3, 50.20000000000001, 52.199999999999996, 51.1, 50.9, 50.20000000000001], [51.1, 50.20000000000001, 51.1, 50.00000000000001, 51.1, 50.20000000000001, 50.00000000000001, 50.70000000000001, 50.20000000000001, 50.20000000000001, 51.0, 52.0, 51.10000000000001, 52.8, 51.50000000000001, 50.20000000000001, 52.199999999999996, 50.20000000000001, 50.20000000000001, 51.1], [51.1, 50.20000000000001, 51.0, 50.00000000000001, 52.199999999999996, 50.20000000000001, 50.00000000000001, 50.20000000000001, 50.20000000000001, 50.20000000000001, 50.20000000000001, 51.10000000000001, 51.1, 50.20000000000001, 51.50000000000001, 50.20000000000001, 52.300000000000004, 50.20000000000001, 50.9, 50.20000000000001], [51.1, 50.20000000000001, 50.20000000000001, 50.00000000000001, 50.20000000000001, 50.20000000000001, 50.00000000000001, 51.0, 50.20000000000001, 50.9, 50.20000000000001, 51.10000000000001, 50.00000000000001, 51.10000000000001, 51.1, 50.20000000000001, 51.1, 51.1, 50.9, 50.20000000000001], [51.1, 50.20000000000001, 50.00000000000001, 51.6, 51.1, 51.0, 50.00000000000001, 50.70000000000001, 50.20000000000001, 50.20000000000001, 50.20000000000001, 51.10000000000001, 50.00000000000001, 50.20000000000001, 50.20000000000001, 50.20000000000001, 50.20000000000001, 50.20000000000001, 50.20000000000001, 50.00000000000001]]


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats


averages = []
conf_intervals = []

for sublist in cost:
    numeric_values = [x for x in sublist if isinstance(x, (int, float))]
    if numeric_values:
        avg = np.mean(numeric_values)
        std_dev = np.std(numeric_values, ddof=1)
        n = len(numeric_values)
        df = n - 1

        t_value = stats.t.ppf(1 - 0.025, df)

        ci = (avg - t_value * std_dev / np.sqrt(n),
              avg + t_value * std_dev / np.sqrt(n))
    else:
        avg = 0
        ci = (0, 0)
    averages.append(avg)
    conf_intervals.append(ci)

lower_bounds = [ci[0] for ci in conf_intervals]
upper_bounds = [ci[1] for ci in conf_intervals]

plt.figure(figsize=(10, 6))
plt.plot(time_window_time_list[6:], averages[6:], linestyle='-', color='blue')
plt.fill_between(time_window_time_list[6:], lower_bounds[6:], upper_bounds[6:], color='blue', alpha=0.2)
plt.xlabel('Duration of the time window [min]')
plt.ylabel('Average cost function value')
plt.grid(True)
plt.legend()
plt.show()
plt.show()